## Data Read

In [ ]:
import pandas as pd
import numpy as np

# Read the data
in_time_data = '/content/drive/MyDrive/workplace-engineering-take-home-test-main/src/in_time.csv'
out_time_data = '/content/drive/MyDrive/workplace-engineering-take-home-test-main/src/out_time.csv'


# Transform data into DataFrame
in_time = pd.read_csv(in_time_data, index_col=0)
out_time = pd.read_csv(out_time_data, index_col=0)

# Add the index name
in_time.index.name = 'EmployeeID'
out_time.index.name = 'EmployeeID'

# Add a higer index name for columns
time_cols = in_time.columns
new_in_cols = []
new_out_cols = []

for col in time_cols:
  new_in_cols.append(("InTime", col))
  new_out_cols.append(("OutTime", col))

in_time.columns = pd.MultiIndex.from_tuples(new_in_cols)
out_time.columns = pd.MultiIndex.from_tuples(new_out_cols)

time_logs = in_time.merge(out_time, on='EmployeeID')

In [ ]:
def weekHours (days_row):
  week_hours = pd.Series()

  index = 1
  for x in range(0, days_row.size, 5):
    curr_week = np.sum(days_row[x:x+5])

    week_hours[f"Week_{index}"] = curr_week

    index += 1

  return week_hours

In [ ]:
def workLogsData(df_time_logs):
  working_days_out_df = pd.DataFrame(df_time_logs.InTime.isna().sum(axis=1), columns=['WorkingDaysOut'])

  in_time_datetimes = df_time_logs.InTime.apply(pd.to_datetime)
  out_time_datetimes = df_time_logs.OutTime.apply(pd.to_datetime)

  time_worked_datedif = out_time_datetimes - in_time_datetimes

  time_worked_hours_df = time_worked_datedif.applymap(lambda x: x.seconds / 3600)

  prom_day_hours_df = pd.DataFrame(time_worked_hours_df.apply(np.mean, axis=1), columns=['PromDayHours'])

  prom_week_hours_df = pd.DataFrame(time_worked_hours_df.apply(weekHours, axis=1), columns=['PromWeekHours'])

  return prom_day_hours_df.merge(working_days_out_df, on='EmployeeID')